In [3]:
import json

In [4]:
with open("/nlsasfs/home/isea/isea10/aansh/introspection/results/single_only.json") as f:
    data = json.load(f)

In [8]:
data["predicted_0"].split("assistant\n\n")[-1]

'Doctor'

In [9]:
gt = []
pred = []
for i in range(80):
    gt.append(data[f"ground_truth_{i}"])
    pred.append(data[f"predicted_{i}"].split("assistant\n\n")[-1])

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(gt, pred)

/nlsasfs/home/isea/isea10/anaconda3/envs/newenv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:98: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_true = type_of_target(y_true, input_name="y_true")
/nlsasfs/home/isea/isea10/anaconda3/envs/newenv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:99: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  type_pred = type_of_target(y_pred, input_name="y_pred")


In [13]:
print(accuracy)

0.6625
